In [ ]:
import argparse
from enum import Enum
from lib2to3.pgen2 import driver
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import time
import random
import pandas as pd

In [ ]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

In [ ]:
PAGINA_URL="https://supermarioplay.com/es"

In [ ]:
#Cargar firefox
from bs4 import BeautifulSoup


driver = webdriver.Firefox()
#Entrar a la pagina especificada
driver.get(PAGINA_URL)
time.sleep(3)
html=driver.page_source
soup=BeautifulSoup(html,'html.parser')

In [ ]:
# WebDriverWait(driver, 10)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'fc-button fc-cta-consent fc-primary-button')]"))).click()

In [ ]:
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
from math import sqrt
import threading
from time import sleep
from selenium.webdriver.common.keys import Keys

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def calculate_distance(puntos,pos1,pos2):
  dist = sqrt((puntos[pos1].x - puntos[pos2].x)**2 + (puntos[pos1].y - puntos[pos2].y)**2)
  return dist

def pintar_texto(annotated_image, text_x, text_y, texto):
  cv2.putText(annotated_image, texto,
          (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
          FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

def draw_landmarks_on_image(rgb_image, detection_result,threadList):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    #Detectar si es piedra, papel o tigera
    puntos = []
    for landmark in hand_landmarks:
      puntos.append(landmark)

    #Cargar una lista de procesos para que no se congele el programa  
    
    actions = webdriver.ActionChains(driver)
    # Pulgar para saltar
    if(calculate_distance(puntos,4,2) > 0.14 and not threadList[0]):
        # print("Pulgar",calculate_distance(puntos,4,2))
        threadList[0]=threading.Thread(target=movimiento,args=(actions,Keys.ARROW_UP,1,threadList,0))
        threadList[0].start()
    # Dedo índice para ir hacia delante
    if (calculate_distance(puntos,8,5) > 0.2 and not threadList[1]): 
        # print("Indice",calculate_distance(puntos,8,5))
        threadList[1]=threading.Thread(target=movimiento,args=(actions,Keys.ARROW_RIGHT,0.01,threadList,1))
        threadList[1].start()
    #Dedo meñique para ir hacia atrás
    if(calculate_distance(puntos,20,17) > 0.13 and not threadList[2]):
        # print("Meñique",calculate_distance(puntos,20,17))
        threadList[2]=threading.Thread(target=movimiento,args=(actions,Keys.ARROW_LEFT,0.01,threadList,2))
        threadList[2].start()
    return annotated_image

def movimiento(actions,key,time,lista,pos):
    actions.key_down(key).perform()
    sleep(time)
    actions.key_up(key).perform()
    lista[pos]=None
      

In [ ]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
import time
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, running_mode=vision.RunningMode.VIDEO,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# Abrir la camara
cap = cv2.VideoCapture(0)

current_timestamp_ms = time.time()
threadList=[None,None,None]

while (cap.isOpened()):
    ret, frame = cap.read()
    img1 = frame

    if ret == True:
        #Mostrar el video
        
        # Bucle lectura imagen
        # STEP 3: Load the input image.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img1)

        # STEP 4: Detect hand landmarks from the input image.
        frame_timestamp_ms = round((time.time() - current_timestamp_ms) * 1000)
        hand_landmarker_result = detector.detect_for_video(mp_image, frame_timestamp_ms)

        # STEP 5: Process the classification result. In this case, visualize it.
        annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), hand_landmarker_result, threadList)
        if annotated_image is None:
            annotated_image = mp_image.numpy_view()
        # plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

        cv2.imshow('pESAO', annotated_image)

        if cv2.waitKey(10) & 0xFF == ord('q'): break
cap.release()
cv2.destroyAllWindows()